In [1]:
import os
import time

import re
import copy
import numpy as np
import pandas as pd
import h5py
import tables
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx
import geopandas as gpd
from shapely.geometry import Point
import rasterio
import rioxarray as rxr
import math
import pickle
from pickle import dump
import sklearn
import graphviz
import xgboost as xgb
import lightgbm as lgbm
from platform import python_version

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from xgboost import cv
from xgboost import XGBRegressor
from xgboost import plot_importance as plot_importance_XGB
from lightgbm import LGBMRegressor
from lightgbm import plot_importance as plot_importance_LGBM

print(pd.__version__) # should be 1.3.0
print(sklearn.__version__) # should be 0.24.1
print(python_version())

1.3.5
1.0.2
3.9.5


In [2]:
os.getcwd()
os.chdir('..')
os.getcwd()
os.chdir('..')
os.getcwd()

'/uufs/chpc.utah.edu/common/home/civil-group1/ItalianAlps/SnowCast/Final_'

In [3]:
mlp_optfeatures = open(os.getcwd()+"/Model/Initial_Models_Final/opt_features_initial.pkl","rb")
mlp_optfeatures = pickle.load(mlp_optfeatures)

lgbm_optfeatures = open(os.getcwd()+"/Model/Model_Calibration/Initial_LGBM/Output/opt_features_initial.pkl","rb")
lgbm_optfeatures = pickle.load(lgbm_optfeatures)

mlp_optfeatures_final = open(os.getcwd()+"/Model/Prev_SWE_Models_Final/opt_features_prevSWE.pkl","rb")
mlp_optfeatures_final = pickle.load(mlp_optfeatures_final)

lgbm_optfeatures_final = open(os.getcwd()+"/Model/Model_Calibration/Prev_LGBM/Output/opt_features_final.pkl","rb")
lgbm_optfeatures_final = pickle.load(lgbm_optfeatures_final)

In [4]:
### define regions
Region_list = ['N_Sierras',
               'S_Sierras_High',
               'S_Sierras_Low',
               'Greater_Yellowstone',
               'N_Co_Rockies',
               'SW_Mont',
               'SW_Co_Rockies',
               'GBasin',
               'N_Wasatch',
               'N_Cascade',
               'S_Wasatch',
               'SW_Mtns',
               'E_WA_N_Id_W_Mont',
               'S_Wyoming',
               'SE_Co_Rockies',
               'Sawtooth',
               'Ca_Coast',
               'E_Or',
               'N_Yellowstone',
               'S_Cascade',
               'Wa_Coast',
               'Greater_Glacier',
               'Or_Coast'
              ]

LGBMregions_intial = ['N_Sierras',
               'S_Sierras_High',
               'S_Sierras_Low',
               'Greater_Yellowstone',
               'GBasin',
               'SE_Co_Rockies',
              ]

LGBMregions_final = ['N_Sierras',
               'S_Sierras_Low',
               'Greater_Yellowstone',
               'N_Co_Rockies',
               'SW_Co_Rockies',
               'GBasin',
               'N_Wasatch',
               'N_Cascade',
               'S_Wasatch',
               'E_WA_N_Id_W_Mont',
               'SE_Co_Rockies',
               'Sawtooth',
               'N_Yellowstone',
               'S_Cascade',
              ]


In [5]:
for region in LGBMregions_intial:
    mlp_optfeatures[region] = lgbm_optfeatures[region]

In [6]:
for region in LGBMregions_final:
    mlp_optfeatures_final[region] = lgbm_optfeatures_final[region]

In [7]:
pickle.dump(mlp_optfeatures, open(os.getcwd()+"/Model/Initial_Models_Final/opt_features_initial.pkl", "wb"))
pickle.dump(mlp_optfeatures_final, open(os.getcwd()+"/Model/Prev_SWE_Models_Final/opt_features_prevSWE.pkl", "wb"))


In [16]:
### Load H5 train files into dictionary
RegionTrain= {}
for region in Region_list:
    RegionTrain[region] = pd.read_hdf('Data/Model_Calibration_Data/RegionTrain_Final.h5', region)
    

In [22]:
#create scaler models from optimal features - final
for region in LGBMregions_intial:
#     ### replace special character ':' with '__' 
#     df = RegionTrain[region][lgbm_optfeatures[region]].rename(columns = lambda x:re.sub(':', '__', x))
    df = RegionTrain[region][lgbm_optfeatures[region]]
    ###normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(df)
    df_scaled = pd.DataFrame(scaled, columns = df.columns)
    #save scaler data here
    pickle.dump(scaler, open(os.getcwd()+'/Model/Model_Calibration/Initial_LGBM/Scalers/'+'LGBM_'+region+'_scaler_new.pkl', 'wb'))

In [10]:
#create scaler models from optimal features - final
for region in LGBMregions_final:
    ###normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(RegionTrain[region][lgbm_optfeatures_final[region]])
    df = pd.DataFrame(scaled, columns = RegionTrain[region][lgbm_optfeatures_final[region]].columns)
    #save scaler data here
    pickle.dump(scaler, open(os.getcwd()+'/Model/Model_Calibration/Prev_LGBM/Scalers/'+'LGBM_'+region+'_scaler_new.pkl', 'wb'))

In [23]:
df

,SWE_SNOTEL:665_NM_SNTL,Prev_SWE_SNOTEL:430_CO_SNTL,SWE_SNOTEL:854_NM_SNTL,SWE_SNOTEL:1005_CO_SNTL,elevation_m,SWE_SNOTEL:1172_NM_SNTL,Lat,SWE_SNOTEL:1254_NM_SNTL,SWE_SNOTEL:1168_NM_SNTL,SWE_SNOTEL:773_CO_SNTL,...,Long,Prev_SWE_SNOTEL:773_CO_SNTL,Prev_SWE_SNOTEL:1168_NM_SNTL,SWE_SNOTEL:532_NM_SNTL,SWE_SNOTEL:744_NM_SNTL,northness,Prev_SWE_SNOTEL:1254_NM_SNTL,Delta_SWE_SNOTEL:430_CO_SNTL,Delta_SWE_SNOTEL:854_NM_SNTL,Delta_SWE_SNOTEL:773_CO_SNTL
1f022144-2f46-41bd-9fa6-e02701cf7843,1.8,-9999.0,3.3,5.4,3247.00000,3.6,38.293376,-9999.0,6.1,5.2,...,-105.850587,-9999.0,-9999.0,5.9,3.0,-0.210817,-9999.0,-9999.99,-9999.99,-9999.99
1f022144-2f46-41bd-9fa6-e02701cf7843,1.7,-9999.0,3.4,6.1,3247.00000,3.9,38.293376,-9999.0,5.9,5.5,...,-105.850587,-9999.0,-9999.0,6.1,3.0,-0.210817,-9999.0,-9999.99,-9999.99,-9999.99
1f022144-2f46-41bd-9fa6-e02701cf7843,1.8,-9999.0,3.5,6.2,3247.00000,3.9,38.293376,-9999.0,6.1,5.6,...,-105.850587,-9999.0,-9999.0,6.4,3.0,-0.210817,-9999.0,-9999.99,-9999.99,-9999.99
1f022144-2f46-41bd-9fa6-e02701cf7843,1.9,-9999.0,4.6,7.0,3247.00000,4.9,38.293376,-9999.0,8.6,6.8,...,-105.850587,-9999.0,-9999.0,8.1,3.5,-0.210817,-9999.0,-9999.99,-9999.99,-9999.99
1f022144-2f46-41bd-9fa6-e02701cf7843,2.7,-9999.0,5.0,7.3,3247.00000,5.0,38.293376,-9999.0,9.1,7.2,...,-105.850587,-9999.0,-9999.0,8.5,4.0,-0.210817,-9999.0,-9999.99,-9999.99,-9999.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNOTEL:1169_NM_SNTL,0.0,0.0,0.0,0.0,3078.47998,0.0,36.787781,0.0,0.1,0.0,...,-105.239990,0.0,4.3,0.0,0.0,-0.075191,0.0,0.00,0.00,0.00
SNOTEL:1169_NM_SNTL,4.0,4.5,6.6,4.0,3078.47998,2.3,36.787781,4.9,7.7,5.7,...,-105.239990,5.1,6.0,4.9,2.5,-0.075191,3.6,1.20,1.30,0.60
SNOTEL:1169_NM_SNTL,4.3,5.7,6.7,5.1,3078.47998,3.3,36.787781,5.1,8.4,7.6,...,-105.239990,5.7,7.7,6.1,2.5,-0.075191,4.9,1.20,0.10,1.90
SNOTEL:1169_NM_SNTL,4.5,6.9,6.8,5.2,3078.47998,3.6,36.787781,5.3,8.6,7.7,...,-105.239990,7.6,8.4,6.4,2.6,-0.075191,5.1,0.20,0.10,0.10


In [17]:
df = RegionTrain[region][lgbm_optfeatures[region]].rename(columns = lambda x:re.sub(':', '__', x))
df

,WYWeek,SWE_SNOTEL__341_OR_SNTL,SWE_SNOTEL__558_OR_SNTL,Prev_SWE_SNOTEL__558_OR_SNTL,SWE_SNOTEL__726_OR_SNTL,Long,Prev_SWE_SNOTEL__341_OR_SNTL,elevation_m,Delta_SWE_SNOTEL__341_OR_SNTL
dd7f64aa-3556-409b-bc86-71dc28b6fd45,14,15.3,11.5,-9999.0,14.4,-123.340785,-9999.0,1620.000000,-9999.99
dd7f64aa-3556-409b-bc86-71dc28b6fd45,15,17.0,13.7,-9999.0,13.9,-123.340785,-9999.0,1620.000000,-9999.99
dd7f64aa-3556-409b-bc86-71dc28b6fd45,16,17.0,13.6,-9999.0,13.7,-123.340785,-9999.0,1620.000000,-9999.99
dd7f64aa-3556-409b-bc86-71dc28b6fd45,17,18.1,13.8,-9999.0,12.6,-123.340785,-9999.0,1620.000000,-9999.99
dd7f64aa-3556-409b-bc86-71dc28b6fd45,18,18.3,14.2,-9999.0,11.2,-123.340785,-9999.0,1620.000000,-9999.99
...,...,...,...,...,...,...,...,...,...
SNOTEL:1084_OR_SNTL,38,0.0,0.0,0.0,0.0,-123.275627,0.0,128.016006,0.00
SNOTEL:1084_OR_SNTL,10,4.9,1.9,3.2,0.0,-123.275627,4.4,128.016006,0.50
SNOTEL:1084_OR_SNTL,11,6.4,1.9,1.9,0.0,-123.275627,4.9,128.016006,1.50
SNOTEL:1084_OR_SNTL,12,6.4,0.6,1.9,0.1,-123.275627,6.4,128.016006,0.00
